In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# <font color='red'>**Loading trained networks**</font>
## Useful functions

In [ ]:
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from numpy import asarray
import tensorflow as tf
AUTOTUNE = tf.data.AUTOTUNE
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
import imageio
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_fscore_support as score
from tensorflow.keras import layers
from sklearn import preprocessing
from tensorflow.keras.utils import plot_model

In [ ]:
!pip install -q git+https://github.com/tensorflow/examples.git

In [ ]:
from tensorflow_examples.models.pix2pix import pix2pix

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256
OUTPUT_CHANNELS = 3

### Import and reuse the Pix2Pix models

In [ ]:
OUTPUT_CHANNELS = 3

#generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
#generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

## Improving the Generator model
### Basic utils

In [ ]:
class InstanceNormalization(tf.keras.layers.Layer):
  """Instance Normalization Layer (https://arxiv.org/abs/1607.08022)."""

  def __init__(self, epsilon=1e-5):
    super(InstanceNormalization, self).__init__()
    self.epsilon = epsilon

  def build(self, input_shape):
    self.scale = self.add_weight(
        name='scale',
        shape=input_shape[-1:],
        initializer=tf.random_normal_initializer(1., 0.02),
        trainable=True)

    self.offset = self.add_weight(
        name='offset',
        shape=input_shape[-1:],
        initializer='zeros',
        trainable=True)

  def call(self, x):
    mean, variance = tf.nn.moments(x, axes=[1, 2], keepdims=True)
    inv = tf.math.rsqrt(variance + self.epsilon)
    normalized = (x - mean) * inv
    return self.scale * normalized + self.offset

In [ ]:
def downsample(filters, size, norm_type='batchnorm', apply_norm=True):
  """Downsamples an input.
  Conv2D => Batchnorm => LeakyRelu
  Args:
    filters: number of filters
    size: filter size
    norm_type: Normalization type; either 'batchnorm' or 'instancenorm'.
    apply_norm: If True, adds the batchnorm layer
  Returns:
    Downsample Sequential Model
  """
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_norm:
    if norm_type.lower() == 'batchnorm':
      result.add(tf.keras.layers.BatchNormalization())
    elif norm_type.lower() == 'instancenorm':
      result.add(InstanceNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [ ]:
def upsample(filters, size, norm_type='batchnorm', apply_dropout=False):
  """Upsamples an input.
  Conv2DTranspose => Batchnorm => Dropout => Relu
  Args:
    filters: number of filters
    size: filter size
    norm_type: Normalization type; either 'batchnorm' or 'instancenorm'.
    apply_dropout: If True, adds the dropout layer
  Returns:
    Upsample Sequential Model
  """

  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                      padding='same',
                                      kernel_initializer=initializer,
                                      use_bias=False))

  if norm_type.lower() == 'batchnorm':
    result.add(tf.keras.layers.BatchNormalization())
  elif norm_type.lower() == 'instancenorm':
    result.add(InstanceNormalization())

  if apply_dropout:
    result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

### Generators

In [ ]:
def unet_generator(img_width, img_high, output_channels, norm_type='instancenorm'):
  """Modified u-net generator model (https://arxiv.org/abs/1611.07004).
  Args:
    output_channels: Output channels
    norm_type: Type of normalization. Either 'batchnorm' or 'instancenorm'.
  Returns:
    Generator model
  """

  down_stack = [
      downsample(64, 4, norm_type, apply_norm=False),  # (bs, 128, 128, 64)
      downsample(128, 4, norm_type),  # (bs, 64, 64, 128)
      downsample(256, 4, norm_type),  # (bs, 32, 32, 256)
      downsample(512, 4, norm_type),  # (bs, 16, 16, 512)
      downsample(512, 4, norm_type),  # (bs, 8, 8, 512)
      downsample(512, 4, norm_type),  # (bs, 4, 4, 512)
      downsample(512, 4, norm_type),  # (bs, 2, 2, 512)
      downsample(512, 4, norm_type),  # (bs, 1, 1, 512)
  ]

  up_stack = [
      upsample(512, 4, norm_type, apply_dropout=True),  # (bs, 2, 2, 1024)
      upsample(512, 4, norm_type, apply_dropout=True),  # (bs, 4, 4, 1024)
      upsample(512, 4, norm_type, apply_dropout=True),  # (bs, 8, 8, 1024)
      upsample(512, 4, norm_type),  # (bs, 16, 16, 1024)
      upsample(256, 4, norm_type),  # (bs, 32, 32, 512)
      upsample(128, 4, norm_type),  # (bs, 64, 64, 256)
      upsample(64, 4, norm_type),  # (bs, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(output_channels, 4, strides=2,
                                         padding='same', kernel_initializer=initializer,
                                         activation='tanh')  # (bs, 256, 256, 3)

  img_input = tf.keras.layers.Input(shape=[img_width, img_high, 3])

  x = img_input

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    #x = concat([x, skip])
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=img_input, outputs=x) #tf.keras.Model(inputs=[img_input, input_label], outputs=x)

In [ ]:
OUTPUT_CHANNELS = 3
generator_g = unet_generator(img_width=IMG_WIDTH, img_high=IMG_HEIGHT, output_channels=OUTPUT_CHANNELS)
generator_f = unet_generator(img_width=IMG_WIDTH, img_high=IMG_HEIGHT, output_channels=OUTPUT_CHANNELS)

## Until here

### Initializing optimizers, generatos and discriminators

In [ ]:
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
checkpoint_path = "../models/cyclegan/preprocessed/mri_to_spect/" 

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                        #    discriminator_x=discriminator_x,
                        #    discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

ckpt.restore(ckpt_manager.latest_checkpoint)
if ckpt_manager.latest_checkpoint:
    print("Restored from {}".format(ckpt_manager.latest_checkpoint))
else:
    print("Initializing from scratch.")

# Making net test

In [ ]:
for layer in generator_g.layers:
    print("layer.name: ", layer.name)
    try:
        print("output shape: ", layer.output_shape)
    except:
        print("no output shape")

In [ ]:
initializer = tf.keras.initializers.Constant(1.)
l1 = generator_g.get_layer(name='concatenate')
print(l1.output_shape)
output = l1.output #l1.get_input_at(0)[0]
print(output.shape)
x = layers.Conv2D(filters=output.shape[-1], kernel_size=(2,2), kernel_initializer=initializer)(output)
x = layers.Reshape((output.shape[-1],))(x)
emb2 = Model(inputs=generator_g.input, outputs=x)
emb2.summary()

# until here

# <font color='red'>**Load and preprocess data**</font>

In [ ]:
str2idx = {
    'control': 0,
    'parkinson': 1
}

idx2str = {
    0: 'control',
    1: 'parkinson'
}

## Helper functions

In [ ]:
def ohe_class(index):
    """
    One-Hot Encoding for Classification Task

    This function takes an integer 'index' representing the class label and performs
    one-hot encoding for a classification task. One-hot encoding is a technique used
    to convert categorical data (class labels) into a binary vector representation,
    where the index of the class label is marked with 1 and all other elements are 0.

    Parameters:
        index (int): An integer representing the class label that needs to be one-hot encoded.
                     It must be a non-negative integer less than the number of classes.

    Returns:
        numpy.ndarray: A NumPy array representing the one-hot encoded label. The length of
                       the array is equal to the number of classes, and the element at the
                       specified 'index' is set to 1, while all other elements are set to 0.

    Example:
        Suppose there are three classes: 0, 1, and 2. To one-hot encode class 1, use:
        >>> class_index = 1
        >>> encoded_label = ohe_class(class_index)
        >>> print(encoded_label)
        Output: [0 1 0]

    Note:
        The function assumes that the number of classes is fixed to 3, as the length of
        the one-hot encoded label is hard-coded to 3. If your classification task involves
        a different number of classes, you will need to modify the function accordingly.
    """
    # Create an array of zeros with length 3 and integer data type
    ohe_label = np.zeros(2, dtype=int)
    
    # Set the element at 'index' to 1 to represent the one-hot encoding
    ohe_label[index] = 1
    
    # Return the one-hot encoded label as an array
    return ohe_label


In [ ]:
# scaling the images to [-1, 1]
def normalize(image):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image

def preprocess_image(image):
    image = normalize(image)
    return image

In [ ]:
def load_images(path, size=(256, 256)):
    """
    Load Images and Corresponding Labels from a Directory into Memory.

    This function loads all images from a specified directory and their corresponding
    labels (assumed to be encoded in the image filenames) into memory. The images are
    loaded, resized to the specified dimensions, and converted into numpy arrays.
    Labels are extracted from the filenames and converted into one-hot encoded vectors.

    Parameters:
        path (str): The path to the directory containing the images.
        size (tuple, optional): A tuple (width, height) specifying the target size
                                for resizing the images. Default is (256, 256).
        rgb (bool, optional): Set to True to load images in RGB color mode,
                              False to load in grayscale mode. Default is False.

    Returns:
        numpy.ndarray: A NumPy array containing the image data. Each element in the
                       array is an image represented as a numpy array.
        list: A list of one-hot encoded labels corresponding to each image in the
              same order as the image data. Each label is represented as a NumPy
              array of length equal to the number of classes.

    Note:
        The function uses Keras' 'load_img' and 'img_to_array' functions to load
        and convert the images. Ensure that Keras or an appropriate library is
        installed before using this function.

    Example:
        >>> data_path = "/path/to/images/"
        >>> image_data, labels = load_images(data_path, size=(128, 128), rgb=True)
        >>> print(image_data.shape)
        Output: (num_images, 128, 128, 3)  # Assuming num_images is the total number of images.
        >>> print(len(labels))
        Output: num_images  # Number of images, each with a corresponding one-hot encoded label.
    """

    data_list = list()
    label_list = list()

    # Enumerate filenames in the directory, assuming all are images
    for filename in tqdm(os.listdir(path)):
        # Load and resize the image
        pixels = load_img(os.path.join(path, filename), target_size=size, color_mode="rgb")
        # Convert to numpy array
        pixels = img_to_array(pixels)

        # Store the image data
        data_list.append(pixels)

        # For labels control: control_case_3811_slice_081.png
        group = filename.split('_')[0]
        # Assuming 'str2idx' is a dictionary mapping class names to their respective indices
        indx = str2idx[group]
        # Get one-hot encoding from the index
        ohe_label = ohe_class(indx)
        label_list.append(ohe_label)

    return np.asarray(data_list), label_list


In [ ]:
def saving_emb(split, group, embeddings, labels, modality=None):
    """
    Save Embeddings, Labels, and Videos to Files.

    This function takes embeddings, labels, and videos obtained from a model and
    saves them to separate files for later use. The data is saved as NumPy arrays.

    Parameters:
        split (str): Indicates the data split, either 'train' or 'test', to determine
                     the destination directory for saving the files.
        clase (str): The class name or identifier to be included in the file names
                     for better organization.
        embeddings (list): A list of embeddings (feature vectors) obtained from a model.
        labels (list): A list of one-hot encoded labels corresponding to the embeddings.
        videos (list): A list of video data associated with the embeddings (optional).

    Note:
        The function converts the input lists 'embeddings', 'labels', and 'videos'
        into NumPy arrays before saving them. Ensure that the data is properly formatted
        before calling this function.
    """
    # Convert the input lists to NumPy arrays
    embeddings_arr = np.array(embeddings)
    labels_arr = np.array(labels)

    print("emb dimension: ", embeddings_arr.shape)
    print("label dimension: ", labels_arr.shape)

    # Create the file path based on the split and class name
    if split == 'train':        
        #file_name = "../embeddings/cyclegan/mri_to_spect/train_" + group + "/"# + clase + '/'
        file_name = "../embeddings/cyclegan/spect_to_mri/train_" + modality + "/"# + clase + '/'
    else:
        file_name = "../embeddings/cyclegan/spect_to_mri/test_" + modality + "/"# + clase + '/'

    print("saving on: ", file_name)

    # Create the directory if it does not exist
    if not os.path.exists(file_name):
        os.makedirs(file_name)

    # Save the embeddings, labels, and videos (if provided) as separate files
    np.save(file_name + "Embeddings", embeddings_arr)
    np.save(file_name + "Labels", labels_arr)

In [ ]:
def toDataSet(path_origen):
    """
    Convert Images and Labels to TensorFlow Dataset.

    This function loads images and corresponding labels from a specified directory,
    converts them into TensorFlow datasets, applies preprocessing to the images,
    and returns a combined dataset containing the image and label pairs.

    Parameters:
        path_origen (str): The path to the directory containing the images.

    Returns:
        tf.data.Dataset: A TensorFlow dataset containing image and label pairs.
                        The images are preprocessed and batched, and the labels
                        are cast to int64 data type.

    Note:
        This function assumes that the 'load_images' function is defined and returns
        a list of image data and labels. It also assumes the availability of 'BATCH_SIZE',
        'AUTOTUNE', and 'BUFFER_SIZE' variables for data preprocessing.

    """
    # Load images and labels using the 'load_images' function
    data, labels = load_images(path_origen)

    # Convert the data to NumPy array
    data_array = np.asarray(data)

    # Create a TensorFlow dataset for the image data
    data_ds = tf.data.Dataset.from_tensor_slices(data_array)

    # Create a TensorFlow dataset for the labels and batch them
    labels_ds = tf.data.Dataset.from_tensor_slices(tf.cast(labels, tf.int64)).batch(BATCH_SIZE)

    # Apply image preprocessing, cache, shuffle, and batch the image dataset
    data_ds = data_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

    # Combine the image and label datasets
    data_label_ds = tf.data.Dataset.zip((data_ds, labels_ds))

    return data_label_ds

# <font color='red'>**Generator embeddings**</font>

In [ ]:
def load_emb_model(generator):
    """
    Load Embedding Model from a Generator Model.

    This function takes a generator model, extracts the intermediate embedding layer,
    and creates a new model (embedding model) that outputs the embeddings obtained
    from the intermediate layer. The function returns this new embedding model.

    Parameters:
        generator_g (tf.keras.Model): The generator model from which to extract
                                      the intermediate embedding layer.

    Returns:
        tf.keras.Model: A new TensorFlow model (embedding model) that takes the same
                        input as the generator model and outputs the embeddings.

    Note:
        This function assumes that the generator model has a layer named 'concatenate'
        representing the intermediate embedding layer.

    """
    print("making emb model")

    initializer = tf.keras.initializers.Constant(1.)


    if generator == "generator_g":
        print("generator_g")
        generator = generator_g
        l1 = generator.get_layer(name='concatenate')
        output = l1.output #l1.get_input_at(0)[0]
        x = layers.Conv2D(filters=output.shape[-1], kernel_size=(2,2), kernel_initializer=initializer)(output)
        x = layers.Reshape((output.shape[-1],))(x)
        emb2 = Model(inputs=generator.input, outputs=x)
    else:
        print("generator_f")
        generator = generator_f
        l1 = generator.get_layer(name='concatenate_7')
        output = l1.output #l1.get_input_at(0)[0]
        x = layers.Conv2D(filters=output.shape[-1], kernel_size=(2,2), kernel_initializer=initializer)(output)
        x = layers.Reshape((output.shape[-1],))(x)
        emb2 = Model(inputs=generator.input, outputs=x)
    
    return emb2

## Generating embeddings

In [ ]:
split = 'train'
group = 'control'
modality = "spect"
task = "mri_to_spect"

if task != "mri_to_spect":
     gen_path = '../data/full_rois/mri/' + split + '_' + group + '/'
     clase = group
else:
     clase = modality
     gen_path = '../data/preprocessed/full_rois/mri_to_spect/' + split + '_' + modality + '/'

#print("gen_path: ", gen_path)
generator = 'generator_f'

#loading embedding model
emb2 = load_emb_model(generator)
data_ds = toDataSet(gen_path)
can = len(data_ds)
print("cantidad: ", can)
embeddings, labels = [], []
# label = [clase]*can
# labels.extend(label)
for img, label in tqdm(data_ds):
     out = emb2(img)#emb2([img])#
     embeddings.extend(out)
     current_label = idx2str[np.argmax(label[0])]
     labels.append(current_label)
saving_emb(split, group, embeddings, labels, modality)

In [ ]:
import numpy as np
data_path = "../embeddings/cyclegan/mri_to_spect/raw/test_mri/Embeddings.npy"
labels_path = "../embeddings/cyclegan/mri_to_spect/raw/test_mri/Labels.npy"

data = np.load(data_path)
labels = np.load(labels_path)
print(data.shape)
print(len(labels))

In [ ]:
data.min()

In [ ]:
data.max()

In [ ]:
type(labels)

In [ ]:
labels

In [ ]:
np.count_nonzero(labels == "control")

In [ ]:
986*2